Author: 

Joel W. Ager (jwager@lbl.gov, [ORCID](https://orcid.org/0000-0001-9334-9751)) [Group website](http://ager.mse.berkeley.edu), [GitHub](https://github.com/joelager)

Jinghan Li (mseljh@sjtu.edu.cn, [ORCID](https://orcid.org/0000-0003-3619-5121))

In [1]:
import os
import glob
from pathlib import Path
import zacros_wrapper as zw
import pandas as pd
import numpy as np
from units import * 
import matplotlib.pyplot as plt

In [2]:
######################### Change these for specific cases #############################
n_trajectory = 10
SitesNum = 40000
######################### Change these for specific cases #############################
notebook_wd = Path('C:/Users/mselj/Desktop/Paper/Paper-KMC/Upload-notebooks+simulation-files')
zacros_relative_file_location = Path('05-Isotope-Multi')
zacros_wd = os.path.join(notebook_wd, zacros_relative_file_location)
print(zacros_wd)
os.chdir(zacros_wd)

multi_1 = zw.KMC_Run.kmc_traj(path=zacros_wd)
result = np.zeros(shape=(n_trajectory,1))
for i in range(n_trajectory):
    print('Starting trajectory', i + 1)
    os.system('wsl.exe zacros-s.x')
    print('finished trajectory:', multi_1.CheckComplete())
    SpecOut = zw.IO_data.SpecnumOut()
    SpecOut.ReadOut(zacros_wd)
    SpecOutN = SpecOut.t.shape[0]
    Temp = SpecOut.t.reshape (SpecOutN,1)
    temp1 = np.concatenate([Temp, SpecOut.spec], axis = 1) # merge the 2 arrays
    df = pd.DataFrame(temp1) # change arrays to Dataframe.
    df.rename(columns={0: 'Time', 1: 'CO*', 2: 'EO*', 3: 'OCCO**', 4: 'OECO**', 5: 'OEEO**', 6: 'H*', 7: 'CCH4', 8: 'ECH4', 9: 'EEH4',\
                       10: 'CCH6O', 11: 'ECH6O', 12: 'EEH6O', }, inplace=True)#E represents 13C, C represents 12C, CCH4 for ethylene, CCH6O for ethanol.
    df['Ethylene'] = df.apply(lambda x: x['CCH4'] + x['ECH4'] + x['EEH4'], axis=1)# sum of all ethylene produced.
    df['Ethanol'] = df.apply(lambda x: x['CCH6O'] + x['ECH6O'] + x['EEH6O'], axis=1)# sum of all ethanol produced.
    df['Iso_ethylene'] = df.apply(lambda x: (x['ECH4'] + 2*x['EEH4'])/(2*x['Ethylene']), axis=1)# 13C ratio in produced ethylene.
    df['Iso_ethanol'] = df.apply(lambda x: (x['ECH6O'] + 2*x['EEH6O'])/(2*x['Ethanol']), axis=1)# 13C ratio in produced ethanol.
    df['c_CO*'] = df.apply(lambda x: x['CO*']/SitesNum, axis=1)
    df['c_EO*'] = df.apply(lambda x: x['EO*']/SitesNum, axis=1)
    df['c_CO*+EO*'] = df.apply(lambda x: (x['CO*']+x['EO*'])/SitesNum, axis=1)
    df['diff'] = df.apply(lambda x: x['Iso_ethylene'] - x['Iso_ethanol'], axis=1)
    df.to_csv(os.path.join(zacros_wd,'coverage {}.csv'.format(i)))
    result[i] = df.iloc[-1,-1] # The difference in 13C fraction in the products at the end of each run. (Only 0.5 s here for demonstration.)
df_diff = pd.DataFrame(result)
df_diff.to_csv(os.path.join(zacros_wd,'diff.csv'))# Save to file.

C:\Users\mselj\Desktop\Paper\Paper-KMC\Upload-notebooks+simulation-files\05-Isotope-Multi
Starting trajectory 1
finished trajectory: True
Starting trajectory 2


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in double_scalars
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in double_scalars


finished trajectory: True
Starting trajectory 3
finished trajectory: True
Starting trajectory 4
finished trajectory: True
Starting trajectory 5
finished trajectory: True
Starting trajectory 6
finished trajectory: True
Starting trajectory 7
finished trajectory: True
Starting trajectory 8
finished trajectory: True
Starting trajectory 9
finished trajectory: True
Starting trajectory 10
finished trajectory: True
